# BATTLE OF NEIGHBOURHOOD - PARIS
## Exploring Paris
### Let's import all the necessary libraries

In [1]:
!pip install geocoder
!pip install bs4
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup

     |████████████████████████████████| 102kB 4.3MB/s ta 0:00:011
     |████████████████████████████████| 92kB 14.9MB/s eta 0:00:01
     |████████████████████████████████| 829kB 19.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
     |███████████████████████████████▎| 112kB 5.9MB/s eta 0:00:01     |████████████████████████████████| 122kB 5.9MB/s 
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


##__Extract Data from Wikepedia__

In [2]:
url="https://en.wikipedia.org/wiki/Arrondissements_of_Paris"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page)
all_tables=soup.find_all("table")
right_table=soup.find('table',class_="wikitable sortable")

In [3]:
A=[]
B=[]
C=[]
D=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==8:
        A.append((cells[0].find(text=True)).strip())
        B.append((cells[1].find(text=True)).strip())
        C.append((cells[2].find(text=True)).strip())
        D.append((cells[4].find(text=True)).strip())
        
import re
A2=[]
subs="("
for r in A:   
    #start=str.find(A[r],"(")
    length=len(r)
    start=str.find(r,"(")
    A2.append(r[0:start-1] + ' Arrondissement')


#create DF
df=pd.DataFrame([A2,B,C,D])
df=df.transpose()
df.columns=(['Arrondissement','Area','SqKM','Population'])
df

,Arrondissement,Area,SqKM,Population
0,1st Arrondissement,Louvre,1.826 km,"17,700"
1,2nd Arrondissement,Bourse,0.992 km,"20,700"
2,3rd Arrondissement,Temple,1.171 km,"35,100"
3,4th Arrondissement,Hôtel-de-Ville,1.601 km,"28,600"
4,5th Arrondissement,Panthéon,2.541 km,"60,600"
5,6th Arrondissement,Luxembourg,2.154 km,"45,200"
6,7th Arrondissement,Palais-Bourbon,4.088 km,"55,400"
7,8th Arrondissement,Élysée,3.881 km,"38,700"
8,9th Arrondissement,Opéra,2.179 km,"58,500"
9,10th Arrondissement,Entrepôt,2.892 km,"88,800"


## Import Geopy and get Latitude and Longitude for all the Arrondissements / areas


In [5]:
!pip install geopy

     |████████████████████████████████| 122kB 5.4MB/s eta 0:00:01


In [6]:
from geopy.geocoders import Nominatim
lat=[]
lon=[]
loc=[]
for r in df.index:
    area=df.iloc[r,0]
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(area)
    loc.append(area)
    lat.append(location.latitude)
    lon.append(location.longitude)


df_longlat=pd.DataFrame([loc,lat,lon])

In [7]:
df_longlat=df_longlat.transpose()
df_longlat.columns=(['Arrondissement','Latitude','Longitude'])
df_paris=pd.merge(df,df_longlat,how='inner')


In [8]:
df_paris

,Arrondissement,Area,SqKM,Population,Latitude,Longitude
0,1st Arrondissement,Louvre,1.826 km,"17,700",48.8646,2.3344
1,2nd Arrondissement,Bourse,0.992 km,"20,700",48.8687,2.34169
2,3rd Arrondissement,Temple,1.171 km,"35,100",48.8642,2.36094
3,4th Arrondissement,Hôtel-de-Ville,1.601 km,"28,600",48.8562,2.35562
4,5th Arrondissement,Panthéon,2.541 km,"60,600",48.8461,2.34452
5,6th Arrondissement,Luxembourg,2.154 km,"45,200",48.8504,2.33295
6,7th Arrondissement,Palais-Bourbon,4.088 km,"55,400",48.857,2.3202
7,8th Arrondissement,Élysée,3.881 km,"38,700",43.243,5.37887
8,9th Arrondissement,Opéra,2.179 km,"58,500",48.876,2.33996
9,10th Arrondissement,Entrepôt,2.892 km,"88,800",48.8761,2.35991


In [9]:
df_paris.iloc[7,4]=48.86849652
df_paris.iloc[7,5]=2.30449878

## *Get Venue Details from FourSquare API*

In [10]:
CLIENT_ID = 'HUGNSJUP0RDQJP2MJ3X5IEORXTEAF5HQFCVGSCJFKYYTJVTC' # your Foursquare ID
CLIENT_SECRET = 'HRB5OZH12CJNW3UGWOYMFQAKR1M3GE2OFTKDIQEGT2IY1O1K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius=2000
LIMIT=100
VERSION = '20180605' # Foursquare API version

Your credentails:
CLIENT_ID: HUGNSJUP0RDQJP2MJ3X5IEORXTEAF5HQFCVGSCJFKYYTJVTC
CLIENT_SECRET:HRB5OZH12CJNW3UGWOYMFQAKR1M3GE2OFTKDIQEGT2IY1O1K


In [180]:
import requests

In [11]:
import requests
neighborhood_latitude = [48.8646] # neighborhood latitude value
neighborhood_longitude = [2.3344] # neighborhood longitude value
neighborhood_name = ['1st Arrondissement'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

# get all venues near toronto and create a df

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results
print(url)

Latitude and longitude values of ['1st Arrondissement'] are [48.8646], [2.3344].
https://api.foursquare.com/v2/venues/explore?client_id=HUGNSJUP0RDQJP2MJ3X5IEORXTEAF5HQFCVGSCJFKYYTJVTC&client_secret=HRB5OZH12CJNW3UGWOYMFQAKR1M3GE2OFTKDIQEGT2IY1O1K&ll=[48.8646],[2.3344]&v=20180605&radius=2000&limit=100


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Arrondissement', 
                  'Arrondissement Latitude', 
                  'Arrondissement Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
Paris_venues = getNearbyVenues(names=df_paris['Arrondissement'],
                                   latitudes=df_paris['Latitude'],
                                   longitudes=df_paris['Longitude']
                                  )

1st Arrondissement
2nd Arrondissement
3rd Arrondissement
4th Arrondissement
5th Arrondissement
6th Arrondissement
7th Arrondissement
8th Arrondissement
9th Arrondissement
10th Arrondissement
11th Arrondissement
12th Arrondissement
13th Arrondissement
14th Arrondissement
15th Arrondissement
16th Arrondissement
17th Arrondissement
18th Arrondissement
19th Arrondissement
20th Arrondissement


In [14]:
Paris_venues.shape

(1508, 7)

# Summary of 1508 Venue by Arrondissement

In [15]:
Paris_venues.groupby('Arrondissement').count()

,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Arrondissement,,,,,,
10th Arrondissement,100,100,100,100,100,100
11th Arrondissement,81,81,81,81,81,81
12th Arrondissement,42,42,42,42,42,42
13th Arrondissement,53,53,53,53,53,53
14th Arrondissement,47,47,47,47,47,47
15th Arrondissement,49,49,49,49,49,49
16th Arrondissement,25,25,25,25,25,25
17th Arrondissement,100,100,100,100,100,100
18th Arrondissement,69,69,69,69,69,69


In [16]:
# one hot encoding
paris_onehot = pd.get_dummies(Paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Arrondissement'] = Paris_venues['Arrondissement'] 


## Take the mean of all the arrondissements by venue

In [17]:
paris_grouped = paris_onehot.groupby('Arrondissement').mean().reset_index()
paris_grouped

,Arrondissement,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,10th Arrondissement,0.00,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.010000,0.00,0.000000,0.000000,0.010000,0.020000,0.000000,0.00
1,11th Arrondissement,0.00,0.012346,0.012346,0.000000,0.00,0.000000,0.000000,0.012346,0.000000,...,0.00000,0.00,0.024691,0.00,0.012346,0.012346,0.024691,0.000000,0.012346,0.00
2,12th Arrondissement,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.02381,0.00,0.000000,0.00,0.000000,0.000000,0.023810,0.000000,0.000000,0.00
3,13th Arrondissement,0.00,0.000000,0.000000,0.018868,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.018868,0.00,0.000000,0.037736,0.000000,0.000000,0.000000,0.00
4,14th Arrondissement,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.021277,0.00,0.000000,0.021277,0.021277,0.021277,0.000000,0.00
5,15th Arrondissement,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020408,...,0.00000,0.00,0.000000,0.00,0.000000,0.020408,0.000000,0.000000,0.000000,0.00
6,16th Arrondissement,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,17th Arrondissement,0.00,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,...,0.01000,0.00,0.010000,0.00,0.000000,0.020000,0.070000,0.000000,0.000000,0.00
8,18th Arrondissement,0.00,0.000000,0.000000,0.000000,0.00,0.014493,0.014493,0.000000,0.000000,...,0.00000,0.00,0.014493,0.00,0.000000,0.000000,0.014493,0.000000,0.000000,0.00
9,19th Arrondissement,0.00,0.000000,0.000000,0.022727,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


## Create a list of top 10 most visited venues in Paris by Arondissement

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Arrondissement']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Arrondissement'] = paris_grouped['Arrondissement']

for ind in np.arange(paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)


,Arrondissement,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10th Arrondissement,French Restaurant,Bistro,Hotel,Coffee Shop,Café,Pizza Place,Bar,Indian Restaurant,Japanese Restaurant,Restaurant
1,11th Arrondissement,French Restaurant,Bar,Café,Cocktail Bar,Restaurant,Bistro,Pastry Shop,Bakery,Italian Restaurant,Wine Bar
2,12th Arrondissement,Hotel,French Restaurant,Supermarket,Bistro,Bakery,Café,Sushi Restaurant,Ice Cream Shop,Bookstore,Farmers Market
3,13th Arrondissement,Hotel,French Restaurant,Thai Restaurant,Indian Restaurant,Bar,Bakery,Gym,Italian Restaurant,Vietnamese Restaurant,Juice Bar
4,14th Arrondissement,French Restaurant,Italian Restaurant,Bakery,Hotel,Bar,Plaza,Bistro,Sushi Restaurant,Pizza Place,Restaurant
5,15th Arrondissement,French Restaurant,Italian Restaurant,Hotel,Coffee Shop,Korean Restaurant,Japanese Restaurant,Lebanese Restaurant,Bar,Supermarket,Moroccan Restaurant
6,16th Arrondissement,Bakery,French Restaurant,Restaurant,Hotel,Café,Garden,Diner,Sandwich Place,Chinese Restaurant,Supermarket
7,17th Arrondissement,French Restaurant,Bar,Wine Bar,Italian Restaurant,Hotel,Thai Restaurant,Japanese Restaurant,Creperie,Pizza Place,Pub
8,18th Arrondissement,Bar,French Restaurant,Italian Restaurant,Middle Eastern Restaurant,Bistro,Pizza Place,Sandwich Place,Plaza,Coffee Shop,Café
9,19th Arrondissement,Bistro,Bar,Café,Beer Bar,French Restaurant,Italian Restaurant,Brewery,Gastropub,Hotel,Concert Hall


## Clustering using KMeans. 4 Clusters

In [21]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

paris_grouped_clustering = paris_grouped.drop('Arrondissement', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 2, 3, 1, 1, 2, 3, 3, 3], dtype=int32)

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = df_paris

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
paris_merged = paris_merged.join(neighborhoods_venues_sorted.set_index('Arrondissement'), on='Arrondissement')
#paris_merged.head(10) # check the last columns!
paris_merged.shape

(20, 17)

In [23]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## View the clusters on Choropleth Map. use Folium library.

In [25]:
import folium

# create map
map_clusters = folium.Map(location=[lat,lon],zoom_start=12)
                          
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Arrondissement'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
paris_merged.groupby('Cluster Labels').count()

,Arrondissement,Area,SqKM,Population,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,
0,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
1,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


# That's all folks!!